In [4]:
import numpy as np
import random

In [5]:
A = np.array([[1, 2, 4, 8], [1, 3, 9, 27], [
             1, 4, 16, 64], [1, 5, 25, 125]], dtype=float)
A

array([[  1.,   2.,   4.,   8.],
       [  1.,   3.,   9.,  27.],
       [  1.,   4.,  16.,  64.],
       [  1.,   5.,  25., 125.]])

In [6]:
def elim(A, r, c):
    B = np.outer(A[r, :], A[:, c])/A[r, c]
    return B, A-B

In [146]:
elim(A, 0, 1)

(array([[ 1. ,  1.5,  2. ,  2.5],
        [ 2. ,  3. ,  4. ,  5. ],
        [ 4. ,  6. ,  8. , 10. ],
        [ 8. , 12. , 16. , 20. ]]),
 array([[  0. ,   0.5,   2. ,   5.5],
        [ -1. ,   0. ,   5. ,  22. ],
        [ -3. ,  -2. ,   8. ,  54. ],
        [ -7. ,  -7. ,   9. , 105. ]]))

In [14]:
def brute_elimination(A):
    n = A.shape[0]

    for i in range(n):
        # eliminate
        u, v, a = A[i+1:, i], A[i, i+1:], A[i, i]
        B = np.outer(u, v)/a
        A[i+1:, i+1:] -= B

        # cleanup
        A[i, i+1:] = 0
        A[i+1:, i] = 0

    return A

In [94]:
def find_j(i):
    j, j2 = 0, 1
    while i % (j2*2) == 0:
        j, j2 = j+1, j2*2
    return j, j2


def update(r1, r2, c1, c2, acc):
    '''
    ret: (r2-r1+1) x (c2-c1+1) matrix
    '''
    k = len(acc)
    print("UPDATE", acc)

    U = np.ndarray((r2-r1+1, k))
    V = np.ndarray((k, c2-c1+1))
    for i, (u, v, a) in enumerate(acc):
        U[:, i] = u[r1:r2+1]/a
        # print(V[i, :].shape, c1, c2+1, v)
        V[i, :] = v[c1:c2+1]

    return np.matmul(U, V)

In [41]:
def simple_elimination(A):
    n = A.shape[0]
    lazy = [None for x in range(n)]

    for i in range(n-1):
        j, j2 = find_j(i+1)

        # lazy eliminate
        u, v, a = A[:, i].copy(), A[i, :].copy(), A[i, i]
        lazy[i] = (u, v, a)

        # update
        acc = lazy[max(0, i-j2+1):i+1]

        r1, r2, c1, c2 = i+1, min(i+j2, n-1), i+1, n-1
        B = update(r1, r2, c1, c2, acc)
        A[r1:r2+1, c1:c2+1] -= B

        r1, r2, c1, c2 = i+j2+1, n-1, i+1, min(i+j2, n-1)
        if r1 < n:
            B = update(r1, r2, c1, c2, acc)
            A[r1:r2+1, c1:c2+1] -= B

        # cleanup
        A[i, i+1:] = 0.0
        A[i+1:, i] = 0.0

    return A

In [67]:
def complex_brute_elimination(A):
    n = A.shape[0]
    lazy = [None for x in range(n)]

    rows = [*range(n)]
    # random.shuffle(rows)

    for r, c in zip(rows, range(n)):
        j, j2 = find_j(c+1)

        # lazy eliminate
        Arc = A[r, c]
        lazy[c] = (A[:, c].copy(), Arc)

        # update
        for i in range(c-j2+1, c+1):
            v, u, a = A[i, c+1:c+j2+1], *lazy[i]
            B = np.outer(u, v)/a
            A[:, c+1:c+j2+1] -= B

        A[:, c] = 0.0
        A[r, c] = Arc

    return A

In [110]:
def complex_elimination(A):
    n = A.shape[0]
    lazy = [None for x in range(n)]

    rows = [*range(n)]
    # random.shuffle(rows)

    for r, c in zip(rows, range(n)):
        j, j2 = find_j(c+1)

        # lazy eliminate
        Arc = A[r, c]
        lazy[c] = (A[:, c].copy(), Arc)

        # lazy update
        super_lazy = [None for x in range(j2)]
        print("LAZY UPDATE")
        print(A)
        for i in range(j2):
            l, l2 = find_j(i+1)

            # lazy eliminate
            v, u, a = A[c-j2+1+i, :].copy(), *lazy[c-j2+1+i]
            # print(v)
            acc = super_lazy[max(0, i-l2+1):i]+[(u,v,a)]

            # update
            r1, r2, c1, c2 = 0, n-1, c+i+1, min(c+j2, n-1)
            if c+i+1 < n:
                B = update(r1, r2, c1, c2, acc)
                A[r1:r2+1, c1:c2+1] -= B

                print("UPDATE", i)
                print(A)
            
            v, u, a = A[c-j2+1+i, :].copy(), *lazy[c-j2+1+i]
            super_lazy[i] = (u, v, a)

        A[:, c] = 0.0
        A[r, c] = Arc
        print("RESULT")
        print(A)

    return A

In [111]:
complex_elimination(A.copy())

LAZY UPDATE
[[  1.   2.   4.   8.]
 [  1.   3.   9.  27.]
 [  1.   4.  16.  64.]
 [  1.   5.  25. 125.]]
UPDATE [(array([1., 1., 1., 1.]), array([1., 2., 4., 8.]), 1.0)]
UPDATE 0
[[  1.   0.   4.   8.]
 [  1.   1.   9.  27.]
 [  1.   2.  16.  64.]
 [  1.   3.  25. 125.]]
RESULT
[[  1.   0.   4.   8.]
 [  0.   1.   9.  27.]
 [  0.   2.  16.  64.]
 [  0.   3.  25. 125.]]
LAZY UPDATE
[[  1.   0.   4.   8.]
 [  0.   1.   9.  27.]
 [  0.   2.  16.  64.]
 [  0.   3.  25. 125.]]
UPDATE [(array([1., 1., 1., 1.]), array([1., 0., 4., 8.]), 1.0)]
UPDATE 0
[[  1.   0.   0.   0.]
 [  0.   1.   5.  19.]
 [  0.   2.  12.  56.]
 [  0.   3.  21. 117.]]
UPDATE [(array([1., 1., 1., 1.]), array([1., 0., 0., 0.]), 1.0), (array([0., 1., 2., 3.]), array([ 0.,  1.,  5., 19.]), 1.0)]
UPDATE 1
[[ 1.  0.  0.  0.]
 [ 0.  1.  5.  0.]
 [ 0.  2. 12. 18.]
 [ 0.  3. 21. 60.]]
RESULT
[[ 1.  0.  0.  0.]
 [ 0.  1.  5.  0.]
 [ 0.  0. 12. 18.]
 [ 0.  0. 21. 60.]]
LAZY UPDATE
[[ 1.  0.  0.  0.]
 [ 0.  1.  5.  0.]
 [ 0.  0. 

array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. , 12. ,  0. ],
       [ 0. ,  0. ,  0. , 28.5]])

In [26]:
complex_brute_elimination(A.copy())

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 6.]])

In [42]:
simple_elimination(A.copy())

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 6.]])

In [16]:
brute_elimination(A.copy())

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 6.]])

In [152]:
B = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
v = np.array([11, 12, 13])
B[:, 0] = v
B

array([[11,  2,  3],
       [12,  5,  6],
       [13,  8,  9]])

In [153]:
A[0, :]

array([1., 2., 4., 8.])